# Project 4: Web Scraping Job Postings

## B. DATA PREPARATION

In [1]:
import glob
import pandas as pd
import re
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from collections import Counter

pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 200)

states = [ "AK",
                      "AL",
                      "AR",
                      "AS",
                      "AZ",
                      "CA",
                      "CO",
                      "CT",
                      "DC",
                      "DE",
                      "FL",
                      "GA",
                      "GU",
                      "HI",
                      "IA",
                      "ID",
                      "IL",
                      "IN",
                      "KS",
                      "KY",
                      "LA",
                      "MA",
                      "MD",
                      "ME",
                      "MI",
                      "MN",
                      "MO",
                      "MS",
                      "MT",
                      "NC",
                      "ND",
                      "NE",
                      "NH",
                      "NJ",
                      "NM",
                      "NV",
                      "NY",
                      "OH",
                      "OK",
                      "OR",
                      "PA",
                      "PR",
                      "RI",
                      "SC",
                      "SD",
                      "TN",
                      "TX",
                      "UT",
                      "VA",
                      "VI",
                      "VT",
                      "WA",
                      "WI",
                      "WV",
                      "WY"]

In [2]:
all_csv = glob.glob("*.csv")
jobs_df = pd.DataFrame()
list_ = []
for file_ in all_csv:
    df = pd.read_csv(file_, index_col=None, header=0)
    list_.append(df)
jobs_df = pd.concat(list_)

In [3]:
jobs_df.head()

,Unnamed: 0,category,company,degree,details,employment_type,experience,industry,location,salary,title,travel
0,0,Information Technology,OPTOMI,NaN,"\nSenior Data Architect (AWS)\nOptomi, in part...",Full-Time,Experience - 8 to greater than 15 years,Banking - Financial Services,"921 East Fort AvenueBaltimore, MD 21230","$130,000.00 - $200,000.00 /Year",Senior Data Architect (AWS),Travel - None
1,1,"Information Technology, Banking",Information Technology Group LLC,NaN,\nContract Only\n\n\nJob Summary:\nThis positi...,Contractor,Experience - At least 6 year(s),Banking - Financial Services,"Kansas City, MO",NaN,Data Architect,NaN
2,2,Consultant,FRG Technology Consulting,Degree - 2 Year Degree,\nAzure Data Architect-8 Months-90% Remote-100...,Contractor,Experience - At least 3 year(s),Consulting,"Columbus, IN",$100.00 - $130.00 /Hour,Azure Data Architect-8 Months-90% Remote-100-1...,NaN
3,3,"Engineering, Information Technology, Design",Robert Half Technology,NaN,\nRef ID: 03510-9501197370Classification: Data...,Full-Time,65 years,"Computer Hardware, Computer Software","Bartlesville, OK 74003",$48.00 - $60.00 /Hour,Data Architect,NaN
4,4,Insurance,BCforward,Degree - None,\nHighly desired experience with managing and ...,Contractor,Experience - 12 to greater than 15 years,Healthcare - Health Services,"Atlanta, GA",$75.00 /Hour,Database Architect,Travel - None


In [4]:
jobs_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4200 entries, 0 to 299
Data columns (total 12 columns):
Unnamed: 0         4200 non-null int64
category           4148 non-null object
company            3924 non-null object
degree             2134 non-null object
details            4199 non-null object
employment_type    4193 non-null object
experience         3283 non-null object
industry           4199 non-null object
location           4197 non-null object
salary             1919 non-null object
title              4199 non-null object
travel             881 non-null object
dtypes: int64(1), object(11)
memory usage: 426.6+ KB


In [5]:
# Drop "Unnamed: 0" column:
jobs_df = jobs_df.drop(columns="Unnamed: 0")

# Drop duplicate rows:
jobs_df = jobs_df.drop_duplicates()

# Drop the row without 'title' value:
jobs_df = jobs_df.dropna(subset=["title"])

# Drop the row which has "$20,000.00 commission" as the value in 'salary':
jobs_df = jobs_df[jobs_df["salary"] != "$20,000.00 commission"]

# Check number of rows now:
jobs_df.info()

# - Deal with other null values after cleaning up columns individually

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2807 entries, 0 to 299
Data columns (total 11 columns):
category           2773 non-null object
company            2643 non-null object
degree             1387 non-null object
details            2807 non-null object
employment_type    2802 non-null object
experience         2159 non-null object
industry           2807 non-null object
location           2805 non-null object
salary             1089 non-null object
title              2807 non-null object
travel             520 non-null object
dtypes: object(11)
memory usage: 263.2+ KB


In [6]:
# Inspect 'category' unique values:
job_categories = jobs_df['category'].unique()
unique_categories = []
for category_tag in job_categories:
    if type(category_tag) == str:
        categories_list = category_tag.split(", ")
        for category in categories_list:
            unique_categories.append(category.lower())
unique_categories = list(set(unique_categories))

print("Number of unique categories: {}".format(len(unique_categories)))
print("***")

# Encode 'category' to dummy variables manually:
categories_dict = {"cat_" + category: [] for category in unique_categories}

for each_row in jobs_df['category']:
    all_categories = [category for category in unique_categories]
    if type(each_row) == str:
        categories_list = each_row.split(", ")
        for category in categories_list:
            categories_dict["cat_" + category.lower()].append(1)
            all_categories.remove(category.lower())
        for category in all_categories:
            categories_dict["cat_" + category.lower()].append(0)
    else:
        for value in categories_dict.values():
            value.append(0)

for key, value in categories_dict.items():
    jobs_df[key] = value

jobs_df = jobs_df.drop(columns="category")
jobs_df.info()

Number of unique categories: 56
***
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2807 entries, 0 to 299
Data columns (total 66 columns):
company                               2643 non-null object
degree                                1387 non-null object
details                               2807 non-null object
employment_type                       2802 non-null object
experience                            2159 non-null object
industry                              2807 non-null object
location                              2805 non-null object
salary                                1089 non-null object
title                                 2807 non-null object
travel                                520 non-null object
cat_construction                      2807 non-null int64
cat_consultant                        2807 non-null int64
cat_pharmaceutical                    2807 non-null int64
cat_media - journalism - newspaper    2807 non-null int64
cat_facilities                       

In [7]:
# Inspect 'industry' unique values:
job_industries = jobs_df['industry'].unique()
unique_industries = []
for industry_tag in job_industries:
    if type(industry_tag) == str:
        industries_list = industry_tag.split(", ")
        for industry in industries_list:
            unique_industries.append(industry.lower())
unique_industries = list(set(unique_industries))

print("Number of unique industries: {}".format(len(unique_industries)))
print("***")

# Encode 'industry' to dummy variables manually:
industries_dict = {"indus_" + industry: [] for industry in unique_industries}

for each_row in jobs_df['industry']:
    all_industries = [industry for industry in unique_industries]
    if type(each_row) == str:
        industries_list = each_row.split(", ")
        for industry in industries_list:
            industries_dict["indus_" + industry.lower()].append(1)
            all_industries.remove(industry.lower())
        for industry in all_industries:
            industries_dict["indus_" + industry.lower()].append(0)
    else:
        for value in industries_dict.values():
            value.append(0)

for key, value in industries_dict.items():
    jobs_df[key] = value

jobs_df = jobs_df.drop(columns="industry")
jobs_df.info()

Number of unique industries: 67
***
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2807 entries, 0 to 299
Columns: 132 entries, company to indus_industrial
dtypes: int64(123), object(9)
memory usage: 2.8+ MB


In [8]:
# Inspect 'company' unique values:
unique_companies = list(set([company.lower() for company in jobs_df['company'].unique() if type(company) == str]))

print("Number of unique companies: {}".format(len(unique_companies)))
print("***")

# Encode 'company' to dummy variables manually:
companies_dict = {"comp_" + company: [] for company in unique_companies}

for company in jobs_df['company']:
    all_companies = [company for company in unique_companies]
    if type(each_row) == str:
        companies_dict["comp_" + company.lower()].append(1)
        all_companies.remove(company.lower())
        for remaining_company in all_companies:
            companies_dict["comp_" + remaining_company.lower()].append(0)
    else:
        for value in companies_dict.values():
            value.append(0) 

for key, value in companies_dict.items():
    jobs_df[key] = value

jobs_df = jobs_df.drop(columns="company")
jobs_df.info()

Number of unique companies: 951
***
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2807 entries, 0 to 299
Columns: 1082 entries, degree to comp_3coast
dtypes: int64(1074), object(8)
memory usage: 23.2+ MB


In [9]:
# Inspect 'degree' unique values:
unique_degrees = []
for degree_tag in jobs_df['degree'].unique():
    if type(degree_tag) == str:
        degrees_list = degree_tag.split(" - ")
        unique_degrees.append(degrees_list[1].lower())
unique_degrees = list(set(unique_degrees))

# Encode 'degree' to dummy variables manually:
degrees_dict = {"deg_" + degree.lower(): [] for degree in unique_degrees}

for company in jobs_df['degree']:
    all_degrees = [degree.lower() for degree in unique_degrees]
    if type(each_row) == str:
        degrees_list = degree_tag.split(" - ")
        degree = degrees_list[1].lower()
        degrees_dict["deg_" + degree].append(1)
        all_degrees.remove(degree)
        for degree in all_degrees:
            degrees_dict["deg_" + degree].append(0)
    else:
        degrees_dict["deg_none"].append(1)
        all_degrees.remove("deg_none")
        for degree in all_degrees:
            degrees_dict["deg_" + degree].append(0)

for key, value in degrees_dict.items():
    jobs_df[key] = value

jobs_df = jobs_df.drop(columns="degree")
jobs_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2807 entries, 0 to 299
Columns: 1087 entries, details to deg_2 year degree
dtypes: int64(1080), object(7)
memory usage: 23.3+ MB


In [10]:
# Inspect 'employment_type' unique values:
unique_types = list(set([employ_type.lower() for employ_type in jobs_df['employment_type'].unique() if type(employ_type) == str]))

print("Number of unique employment types: {}".format(len(unique_types)))
print("***")

# Encode 'employment_type' to dummy variables manually:
types_dict = {"type_" + employ_type: [] for employ_type in unique_types}

for employ_type in jobs_df['employment_type']:
    all_types = [employ_type for employ_type in unique_types]
    if type(employ_type) == str:
        types_dict["type_" + employ_type.lower()].append(1)
        all_types.remove(employ_type.lower())
        for remaining_type in all_types:
            types_dict["type_" + remaining_type.lower()].append(0)
    else:
        for value in types_dict.values():
            value.append(0) 

for key, value in companies_dict.items():
    jobs_df[key] = value

jobs_df = jobs_df.drop(columns="employment_type")
jobs_df.info()

Number of unique employment types: 5
***
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2807 entries, 0 to 299
Columns: 1086 entries, details to deg_2 year degree
dtypes: int64(1080), object(6)
memory usage: 23.3+ MB


In [11]:
# Inspect 'travel' unique values:
unique_travels = list(set([travel.lower() for travel in jobs_df['travel'].unique() if type(travel) == str]))

print("Number of unique travel requirements: {}".format(len(unique_travels)))
unique_travels

Number of unique travel requirements: 7


['travel - none',
 'travel',
 'travel - negligible',
 'travel expenses',
 'travel - up to 50%',
 'travel - up to 25%',
 'travel - road warrior']

In [12]:
jobs_df.loc[jobs_df["travel"] == "Travel", ["travel"]] = "travel - some"
jobs_df.loc[jobs_df["travel"] == "Travel Expenses", ["travel"]] = None

def get_split_value(string):
    if type(string) == str:
        return string.split(" - ")[-1].lower()
    else:
        return string

jobs_df["travel"] = jobs_df["travel"].apply(get_split_value)

# Inspect 'travel' unique values:
unique_travels = list(set([travel for travel in jobs_df['travel'].unique() if type(travel) == str]))

print("Number of unique travel requirements: {}".format(len(unique_travels)))
unique_travels

Number of unique travel requirements: 6


['none', 'up to 25%', 'some', 'negligible', 'up to 50%', 'road warrior']

In [13]:
# Encode 'travel' to dummy variables manually:
travel_dict = {"travel_" + req: [] for req in unique_travels}

for req in jobs_df['travel']:
    all_reqs = [req for req in unique_travels]
    if type(req) == str:
        travel_dict["travel_" + req].append(1)
        all_reqs.remove(req)
        for remaining_req in all_reqs:
            travel_dict["travel_" + remaining_req].append(0)
    else:
        for value in travel_dict.values():
            value.append(0) 

for key, value in travel_dict.items():
    jobs_df[key] = value

jobs_df = jobs_df.drop(columns="travel")
jobs_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2807 entries, 0 to 299
Columns: 1091 entries, details to travel_negligible
dtypes: int64(1086), object(5)
memory usage: 23.4+ MB


In [14]:
# Inspect 'location' unique values:
unique_locations = list(set([location for location in jobs_df['location'].unique() if type(location) == str]))

print("Number of unique locations: {}".format(len(unique_locations)))

# Encode 'location' to dummy variables manually, based on states:
states_dict = {"state_" + state: [] for state in states}

for location in jobs_df['location']:
    if type(location) == str:
        for state in states:
            if state in location:
                states_dict["state_" + state].append(1)
            else:
                states_dict["state_" + state].append(0)
    else:
        for state in states:
            states_dict["state_" + state].append(0)

for key, value in states_dict.items():
    jobs_df[key] = value

jobs_df = jobs_df.drop(columns="location")
jobs_df.info()

Number of unique locations: 1213
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2807 entries, 0 to 299
Columns: 1145 entries, details to state_WV
dtypes: int64(1141), object(4)
memory usage: 24.5+ MB


In [15]:
# Inspect 'salary' unique values:
print(len(jobs_df['salary'].unique()))
jobs_df['salary'].unique()[:10]

520


array(['$130,000.00 - $200,000.00 /Year', nan, '$100.00 - $130.00 /Hour',
       '$48.00 - $60.00 /Hour', '$75.00 /Hour',
       '$155,000.00 - $185,000.00 /Year',
       '$130,000.00 - $150,000.00 /Year',
       '$115,000.00 - $125,000.00 /Year',
       '$170,000.00 - $180,000.00 /Year',
       '$130,000.00 - $170,000.00 /Year'], dtype=object)

In [16]:
# Convert:
# - hourly salary to yearly salary
# - salary range to max, min and average salary

annual_work_hours = 2008 #Taken from https://www.calendar-12.com/working_days/2018

salary_dict = {
    "salary_min": [],
    "salary_max": [],
    "salary_avg": []
}

def get_salary_values(salary_range): 
    if type(salary_range) == str:
        if 'Year' in salary_range:
            sal = re.findall(r'[0-9.]+', salary_range.replace('$','').replace(',',''))
            sal = [float(element) for element in sal]
#             print sal
            if len(sal) == 2:
                salary_dict["salary_min"].append(sal[0])
                salary_dict["salary_max"].append(sal[1])
                salary_dict["salary_avg"].append(np.mean([sal[0], sal[1]]))
            else:
                salary_dict["salary_min"].append(sal[0])
                salary_dict["salary_max"].append(sal[0])
                salary_dict["salary_avg"].append(sal[0])
                
        elif 'Hour' in salary_range:
            sal = re.findall(r'[0-9.]+',salary_range.replace('$','').replace(',',''))
            sal = [float(element) for element in sal]
#             print sal
            if len(sal) == 2:
                salary_dict["salary_min"].append(sal[0] * annual_work_hours)
                salary_dict["salary_max"].append(sal[1] * annual_work_hours)
                salary_dict["salary_avg"].append(np.mean([sal[0], sal[1]]) * annual_work_hours)
            else:
                salary_dict["salary_min"].append(sal[0] * annual_work_hours)
                salary_dict["salary_max"].append(sal[0] * annual_work_hours)
                salary_dict["salary_avg"].append(sal[0] * annual_work_hours)
        else:
            print salary_range
        
    else:
#         print salary_range
        salary_dict["salary_min"].append(None)
        salary_dict["salary_max"].append(None)
        salary_dict["salary_avg"].append(None)


        
for salary_range in jobs_df['salary']:
    get_salary_values(salary_range)

for key, value in salary_dict.items():
    jobs_df[key] = value
    
jobs_df = jobs_df.drop(columns="salary")

jobs_df.info()

# for key, value in salary_dict.items():
#     print "length of {}: {}".format(key, len(value))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2807 entries, 0 to 299
Columns: 1147 entries, details to salary_max
dtypes: float64(3), int64(1141), object(3)
memory usage: 24.6+ MB


In [17]:
# function returns mid point of a given experience range, else pass if no data available
# values above 15 are then removed as they are likely to refer to company age
def experience_cleaner(exp_range):
    if type(exp_range) == str:
        if "None" in exp_range:
            return 0
        else:
            experience = np.mean([float(i) for i in re.findall(r'[0-9]+', exp_range)])
            if experience > 15:
                return np.NaN
            else:
                return experience
    else:
        return np.NaN

jobs_df["experience"] = jobs_df["experience"].apply(experience_cleaner)

In [18]:
# Use the CountVectorizer to find unigrams for most common job titles: 
vect = CountVectorizer()

# Pulls all of job titles into one giant string
title_string = "".join(jobs_df['title'])
title_unigrams = vect.build_analyzer()(title_string)

Counter(title_unigrams).most_common(50)

[(u'data', 460),
 (u'analyst', 307),
 (u'business', 252),
 (u'intelligence', 150),
 (u'engineer', 144),
 (u'developer', 140),
 (u'analystbusiness', 122),
 (u'database', 122),
 (u'analystdata', 115),
 (u'bi', 99),
 (u'scientist', 92),
 (u'software', 70),
 (u'analystsenior', 68),
 (u'sql', 66),
 (u'scientistdata', 64),
 (u'administrator', 59),
 (u'and', 57),
 (u'senior', 49),
 (u'architect', 49),
 (u'net', 44),
 (u'engineerdata', 43),
 (u'engineersenior', 42),
 (u'developerdatabase', 41),
 (u'dba', 41),
 (u'financial', 40),
 (u'server', 40),
 (u'analystsr', 36),
 (u'oracle', 35),
 (u'programmer', 32),
 (u'of', 30),
 (u'analytics', 29),
 (u'architectdata', 29),
 (u'developersenior', 28),
 (u'analystfinancial', 27),
 (u'or', 26),
 (u'level', 25),
 (u'learning', 25),
 (u'analystit', 25),
 (u'research', 24),
 (u'stack', 24),
 (u'it', 23),
 (u'manager', 23),
 (u'lead', 23),
 (u'big', 22),
 (u'developerbusiness', 22),
 (u'java', 22),
 (u'administratordatabase', 22),
 (u'engineersoftware', 21),

In [19]:
# Convert all job titles to lowercase:
jobs_df["title"] = jobs_df["title"].str.lower()

jobs_df['title_intern'] = np.where(jobs_df['title'].str.contains("intern|internship"), 1, 0)
jobs_df['title_lead'] = np.where(jobs_df['title'].str.contains("lead"), 1, 0)
jobs_df['title_principal'] = np.where(jobs_df['title'].str.contains("principal"), 1, 0)
jobs_df['title_chief'] = np.where(jobs_df['title'].str.contains("chief"), 1, 0)
jobs_df['title_senior'] = np.where(jobs_df['title'].str.contains("senior|sr"), 1, 0)
jobs_df['title_junior'] = np.where(jobs_df['title'].str.contains("junior|jr"), 1, 0)

jobs_df['title_data'] = np.where(jobs_df['title'].str.contains("data"), 1, 0)
jobs_df['title_ml'] = np.where(jobs_df['title'].str.contains("machine learning|ml"), 1, 0)
jobs_df['title_database'] = np.where(jobs_df['title'].str.contains("database|dba"), 1, 0)
jobs_df['title_ai'] = np.where(jobs_df['title'].str.contains("artificial intelligence|ai"), 1, 0)
jobs_df['title_business'] = np.where(jobs_df['title'].str.contains("business|bi"), 1, 0)
jobs_df['title_software'] = np.where(jobs_df['title'].str.contains("software"), 1, 0)
jobs_df['title_sql'] = np.where(jobs_df['title'].str.contains("sql"), 1, 0)
jobs_df['title_financial'] = np.where(jobs_df['title'].str.contains("financial"), 1, 0)

jobs_df['title_analyst'] = np.where(jobs_df['title'].str.contains("analyst|analytics"), 1, 0)
jobs_df['title_intelligence'] = np.where(jobs_df['title'].str.contains("intelligence|bi"), 1, 0)
jobs_df['title_engineer'] = np.where(jobs_df['title'].str.contains("engineer"), 1, 0)
jobs_df['title_developer'] = np.where(jobs_df['title'].str.contains("developer"), 1, 0)
jobs_df['title_scientist'] = np.where(jobs_df['title'].str.contains("scientist"), 1, 0)
jobs_df['title_administrator'] = np.where(jobs_df['title'].str.contains("administrator|dba"), 1, 0)
jobs_df['title_architect'] = np.where(jobs_df['title'].str.contains("architect"), 1, 0)

jobs_df = jobs_df.drop(columns="title")
jobs_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2807 entries, 0 to 299
Columns: 1167 entries, details to title_architect
dtypes: float64(4), int64(1162), object(1)
memory usage: 25.0+ MB


In [20]:
# Clean "details" column by removing formatting characters:
def details_cleaner(string):
    return ''.join(s if ord(s)>31 and ord(s)<126 else ' ' for s in string)

jobs_df["details"] = jobs_df["details"].map(details_cleaner)

In [23]:
# See final cleaned dataframe (not preprocessed yet for NLP)
jobs_df.head()

# Save dataframe to csv to load on other notebook:
jobs_df.to_csv('data-prep.csv', encoding='utf-8')

# - Will do feature selection in modelling part to decrease number of features

In [22]:
jobs_df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2807 entries, 0 to 299
Data columns (total 1167 columns):
details                                                        object
experience                                                     float64
cat_construction                                               int64
cat_consultant                                                 int64
cat_pharmaceutical                                             int64
cat_media - journalism - newspaper                             int64
cat_facilities                                                 int64
cat_science                                                    int64
cat_health care                                                int64
cat_sales                                                      int64
cat_manufacturing                                              int64
cat_insurance                                                  int64
cat_qa - quality control                                     